In [1]:
import pickle
import torch
from torch import nn
import util
import model
import dataset

In [2]:
with open('sec-bert.pkl', 'rb') as file:
    net = pickle.load(file)

In [3]:
batch_size, max_len = 512, 64
train_iter, vocab = util.load_csic_data(batch_size, max_len)

In [4]:
devices = util.try_all_gpus()

In [5]:
def get_bert_encoding(net, tokens_a, tokens_b=None):
    tokens, segments = dataset.get_tokens_and_segments(tokens_a, tokens_b)
    print(f'debug tokens: {tokens}')
    token_ids = torch.tensor(vocab[tokens], device=devices[0]).unsqueeze(0)
    segments = torch.tensor(segments, device=devices[0]).unsqueeze(0)
    valid_len = torch.tensor(len(tokens), device=devices[0]).unsqueeze(0)
    encoded_X, _, _ = net(token_ids, segments, valid_len)
    return encoded_X

In [6]:
tokens_a = ['id', '3', 'nombre', 'vino+rioja', 'precio', '100', 'cantidad', '55', 'b1', 'a%f1adir+al+carrito']
encoded_text = get_bert_encoding(net, tokens_a)
encoded_text_cls = encoded_text[:, 0, :]
encoded_text_crane = encoded_text[:, 2, :]
encoded_text.shape, encoded_text_cls.shape, encoded_text_crane[0][:3]

debug tokens: ['<cls>', 'id', '3', 'nombre', 'vino+rioja', 'precio', '100', 'cantidad', '55', 'b1', 'a%f1adir+al+carrito', '<sep>']


(torch.Size([1, 12, 128]),
 torch.Size([1, 128]),
 tensor([-0.7902,  0.0385, -1.7137], device='cuda:0', grad_fn=<SliceBackward0>))

In [7]:
encoded_text

tensor([[[-0.1031,  0.9614, -1.7827,  ..., -1.9561, -1.1995,  0.6418],
         [-0.7995,  0.9725, -1.7305,  ..., -1.9091, -1.1518,  0.6427],
         [-0.7902,  0.0385, -1.7137,  ..., -1.8587, -1.1395,  0.6754],
         ...,
         [-0.8089,  1.0102, -0.0531,  ..., -1.9355, -1.1740,  0.6623],
         [-0.7473,  1.0272, -0.0338,  ..., -1.8981, -0.0132,  0.0134],
         [-0.7755,  1.0067, -0.0582,  ..., -1.9322, -1.1934, -0.0141]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)